<a href="https://colab.research.google.com/github/HajerBch/Darija-GPT/blob/main/DarijaGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers           # Install Hugging Face Transformers&#8203;:contentReference[oaicite:0]{index=0}
!pip install datasets huggingface_hub  # (Optional) Install Datasets and Hub libs&#8203;:contentReference[oaicite:1]{index=1}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00


In [2]:
!huggingface-cli login  # will prompt for the token&#8203;:contentReference[oaicite:5]{index=5}




    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [3]:
import pandas as pd

splits = {'v1': 'data/v1-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/ayoubkirouane/Algerian-Darija/" + splits["v1"])
text = df['Text'].to_list()

with open('data.txt', 'w') as f:
  f.write('\n'.join(text))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Define a generator function to read text data in batches from a file
def text_iterator(file_path, batch_size=1000):
    # Open the file in read mode with UTF-8 encoding
    with open(file_path, 'r', encoding='utf-8') as file:
        batch = []  # Initialize an empty list to store the batch of lines
        # Iterate over each line in the file
        for line in file:
            batch.append(line.strip())  # Strip any leading/trailing whitespace and add the line to the batch
            # If the batch size is reached, yield the batch and reset the batch list
            if len(batch) == batch_size:
                yield batch
                batch = []
        # Yield any remaining lines in the batch that didn't reach the batch size
        if batch:
            yield batch

# Import the AutoTokenizer class from the transformers library
from transformers import AutoTokenizer

# Load the pre-trained GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Train a new tokenizer on the Algerian Darija text data using the text_iterator function
darija_tokenizer = tokenizer.train_new_from_iterator(text_iterator(file_path="data.txt"),
                                                     vocab_size=25000)

# Save the newly trained Darija tokenizer to a directory
darija_tokenizer.save_pretrained("darija_tokenizer")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

('darija_tokenizer/tokenizer_config.json',
 'darija_tokenizer/special_tokens_map.json',
 'darija_tokenizer/vocab.json',
 'darija_tokenizer/merges.txt',
 'darija_tokenizer/added_tokens.json',
 'darija_tokenizer/tokenizer.json')

In [ ]:
darija_tokenizer = AutoTokenizer.from_pretrained("darija_tokenizer")
tokens = darija_tokenizer.tokenize("واش راك خويا؟")
print(tokens)

['ÙĪØ§Ø´', 'ĠØ±Ø§Ùĥ', 'ĠØ®ÙĪÙĬØ§', 'Ø', 'Ł']


In [ ]:
# Importing necessary libraries
import time
import torch
import torch.nn as nn
from torch.nn import functional as F
from dataclasses import dataclass

# This dictionary contains the configuration parameters for different versions of the GPT-2 model
# The parameters include the number of layers, the number of attention heads, and the embedding dimension
# The numbers provided are for the 'gpt2-large' model
"""
{
    'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
    'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
    'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
    'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
}
"""

# The GPTConfig class is a data class that holds the configuration parameters for the GPT-2 model
# The parameters are initialized with the values for the 'gpt2-large' model
@dataclass
class GPTConfig:
    block_size: int = 1024  # The maximum context length for the model
    vocab_size: int = 50257  # The size of the vocabulary
    n_layer: int = 36  # The number of layers in the model
    n_head: int = 20  # The number of attention heads in each layer
    n_embd: int = 1280  # The embedding dimension

In [ ]:
# The MLP class defines a multi-layer perceptron (MLP) module for the GPT-2 model
class MLP(nn.Module):
    # The __init__ method initializes the MLP module with the given configuration
    def __init__(self, config):
        super().__init__()
        # The first linear layer (c_fc) takes the input embedding and maps it to a higher-dimensional space
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        # The gelu activation function is applied to the output of the first linear layer
        self.gelu = nn.GELU(approximate='tanh')
        # The second linear layer (c_proj) maps the output of the gelu activation function back to the original embedding dimension
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        # The NANOGPT_SCALE_INIT attribute is set to 1 to scale the output of the second linear layer
        self.c_proj.NANOGPT_SCALE_INIT = 1

    # The forward method defines the forward pass of the MLP module
    def forward(self, x):
        # The input is passed through the first linear layer
        x = self.c_fc(x)
        # The output of the first linear layer is passed through the gelu activation function
        x = self.gelu(x)
        # The output of the gelu activation function is passed through the second linear layer
        x = self.c_proj(x)
        # The output of the second linear layer is returned as the output of the MLP module
        return x

In [ ]:
# The CausalSelfAttention class defines a causal self-attention module for the GPT-2 model
class CausalSelfAttention(nn.Module):
    # The __init__ method initializes the causal self-attention module with the given configuration
    def __init__(self, config):
        super().__init__()
        # Assert that the embedding dimension is divisible by the number of attention heads
        assert config.n_embd % config.n_head == 0
        # The c_attn linear layer computes the query, key, and value vectors for all attention heads in batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # The c_proj linear layer computes the output projection of the attention module
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # The NANOGPT_SCALE_INIT attribute is set to 1 to scale the output of the c_proj linear layer
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # The number of attention heads and the embedding dimension are stored as attributes of the module
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    # The forward method defines the forward pass of the causal self-attention module
    def forward(self, x):
        # The input is expected to have shape (B, T, C), where B is the batch size, T is the sequence length, and C is the embedding dimensionality
        B, T, C = x.size()
        # The c_attn linear layer is applied to the input to compute the query, key, and value vectors for all attention heads in batch
        qkv = self.c_attn(x)
        # The query, key, and value vectors are split along the channel dimension
        q, k, v = qkv.split(self.n_embd, dim=2)
        # The key and query vectors are reshaped to have shape (B, nh, T, hs), where nh is the number of attention heads and hs is the head size
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        # The value vectors are reshaped to have shape (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        # The scaled dot-product attention is computed using the query, key, and value vectors
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # Flash attention
        # The output of the attention module is reshaped to have shape (B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        # The output projection is computed using the c_proj linear layer
        y = self.c_proj(y)
        # The output of the causal self-attention module is returned
        return y

In [ ]:
# The DecoderBlock class defines a decoder block for the GPT-2 model
class DecoderBlock(nn.Module):
    # The __init__ method initializes the decoder block with the given configuration
    def __init__(self, config):
        super().__init__()
        # The first layer normalization layer (ln_1) is applied to the input before the causal self-attention module
        self.ln_1 = nn.LayerNorm(config.n_embd)
        # The causal self-attention module (attn) computes the attention weights for the input sequence
        self.attn = CausalSelfAttention(config)
        # The second layer normalization layer (ln_2) is applied to the input before the multi-layer perceptron module
        self.ln_2 = nn.LayerNorm(config.n_embd)
        # The multi-layer perceptron module (mlp) computes the output of the decoder block
        self.mlp = MLP(config)

    # The forward method defines the forward pass of the decoder block
    def forward(self, x):
        # The input is passed through the causal self-attention module, with layer normalization applied to the input before the module
        x = x + self.attn(self.ln_1(x))
        # The output of the causal self-attention module is passed through the multi-layer perceptron module, with layer normalization applied to the input before the module
        x = x + self.mlp(self.ln_2(x))
        # The output of the decoder block is returned
        return x